In [182]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import *

In [183]:
'''Taken from Prof. Zingale's code'''
def print_Hb(H, b):
    N = len(b)

    topl = "\u256d"
    topr = "\u256e"
    side = "\u2502"
    botl = "\u2570"
    botr = "\u256F"
    space = 8*" "

    top_str = topl + N*" {:>7.03f} " + topr + space + topl + " {:6.3f} " + topr
    sid_str = side + N*" {:>7.03f} " + side + space + side + " {:6.3f} " + side
    bot_str = botl + N*" {:>7.03f} " + botr + space + botl + " {:6.3f} " + botr

    print(" ")
    for i in range(N):
        if i == 0:
            pstr = top_str
        elif i == N-1:
            pstr = bot_str
        else:
            pstr = sid_str
        out = tuple(H[i, :]) + (b[i],)
        print(pstr.format(*out))

In [184]:
'''Taken from Prof. Zingale's code'''
def gauss_elim(H, b, quiet=0):
    assert b.ndim == 1, "ERROR: b should be a vector"
    N = len(b)
    # H is square, with each dimension of length N
    assert H.shape == (N, N), "ERROR: H should be square with each dim of same length as b"
    # allocation the solution array
    x = np.zeros((N), dtype=H.dtype)
    # find the scale factors for each row -- this is used when pivoting
    scales = np.max(np.abs(H), 1)
    # keep track of the number of times we swapped rows
    num_row_swap = 0

    if not quiet:
        print_Hb(H, b)

    # main loop over rows
    for k in range(N):
        # find the pivot row based on the size of column k -- only consider
        # the rows beyond the current row
        row_max = np.argmax(H[k:, k]/scales[k:])
        if k > 0:
            row_max += k  # we sliced H from k:, correct for total rows
        # swap the row with the largest scaled element in the current column
        # with the current row (pivot) -- do this with b too!
        if not row_max == k:
            H[[k, row_max], :] = H[[row_max, k], :]
            b[[k, row_max]] = b[[row_max, k]]
            scales[[k, row_max]] = scales[[row_max, k]]
            if not quiet:
                print("pivoted")
            num_row_swap += 1

        # do the forward-elimination for all rows below the current
        for i in range(k+1, N):
            coeff = H[i, k] / H[k, k]

            for j in range(k+1, N):
                H[i, j] += -H[k, j] * coeff
            H[i, k] = 0.0
            b[i] += -coeff * b[k]
        
            # check if the row is all zeros -- singular
            if H[i, :].min() == 0 and H[i, :].max() == 0:
                raise ValueError("matrix is singular")
        if not quiet:
            print_Hb(H, b)
    x[N-1] = b[N-1] / H[N-1, N-1]

    for i in reversed(range(N-1)):
        bsum = b[i]
        for j in range(i+1, N):
            bsum += -H[i, j] * x[j]
        x[i] = bsum / H[i, i]
    return x

In [185]:
''' Solving for vector b given a known matrix H and vector x, H @ x = b'''
n = int(input('Enter the matrix size of the Hilbert matrix: ')) #Enter desired size of Hilbert matrix

#Define the Hilbert Matrix
def hilbert_matrix(n):
    return hilbert(n)
#Define vector x
def vector_x(n):
    return np.arange(n)
#Define vector b
def vector_b(n):
    H = hilbert_matrix(n)
    x = vector_x(n)
    return H @ x

ans = vector_b(n)
print("b =", ans)

b = [2.71666667 2.1        1.72142857 1.46190476 1.2718254 ]


In [186]:
'''Solving for vector x given the known vector b and matrix H from above, H^-1 b = x'''
def sys_sol(n):
    H = hilbert(n)
    print("H =", H)
    b = vector_b(n)
    print("b =", b)
    x = gauss_elim(H.copy(), b.copy())
    return x

solution = sys_sol(n)
print('x =', solution)

H = [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]
b = [2.71666667 2.1        1.72142857 1.46190476 1.2718254 ]
 
╭   1.000    0.500    0.333    0.250    0.200 ╮        ╭  2.717 ╮
│   0.500    0.333    0.250    0.200    0.167 │        │  2.100 │
│   0.333    0.250    0.200    0.167    0.143 │        │  1.721 │
│   0.250    0.200    0.167    0.143    0.125 │        │  1.462 │
╰   0.200    0.167    0.143    0.125    0.111 ╯        ╰  1.272 ╯
 
╭   1.000    0.500    0.333    0.250    0.200 ╮        ╭  2.717 ╮
│   0.000    0.083    0.083    0.075    0.067 │        │  0.742 │
│   0.000    0.083    0.089    0.083    0.076 │        │  0.816 │
│   0.000    0.075    0.083    0.080    0.075 │        │  0.783 │
╰   0.000    0.067    0.076    0.075    0.071 

In [187]:
def convergence(k):
    x = vector_x(k)
    x_tilde = sys_sol(k)
    return np.max(np.abs(x_tilde - x))

for k in range(2,16):
    error = convergence(k)
    print(f"N = {k}: error = {error}")

H = [[1.         0.5       ]
 [0.5        0.33333333]]
b = [0.5        0.33333333]
 
╭   1.000    0.500 ╮        ╭  0.500 ╮
╰   0.500    0.333 ╯        ╰  0.333 ╯
 
╭   1.000    0.500 ╮        ╭  0.500 ╮
╰   0.000    0.083 ╯        ╰  0.083 ╯
 
╭   1.000    0.500 ╮        ╭  0.500 ╮
╰   0.000    0.083 ╯        ╰  0.083 ╯
N = 2: error = 0.0
H = [[1.         0.5        0.33333333]
 [0.5        0.33333333 0.25      ]
 [0.33333333 0.25       0.2       ]]
b = [1.16666667 0.83333333 0.65      ]
 
╭   1.000    0.500    0.333 ╮        ╭  1.167 ╮
│   0.500    0.333    0.250 │        │  0.833 │
╰   0.333    0.250    0.200 ╯        ╰  0.650 ╯
 
╭   1.000    0.500    0.333 ╮        ╭  1.167 ╮
│   0.000    0.083    0.083 │        │  0.250 │
╰   0.000    0.083    0.089 ╯        ╰  0.261 ╯
pivoted
 
╭   1.000    0.500    0.333 ╮        ╭  1.167 ╮
│   0.000    0.083    0.089 │        │  0.261 │
╰   0.000    0.000   -0.006 ╯        ╰ -0.011 ╯
 
╭   1.000    0.500    0.333 ╮        ╭  1.167 ╮
│   0.000 

The error becomes $\mathcal{O}(1)$ at $N = 12$